# Lab 6: Sequence to Sequence Text

##Import Statements

In [72]:
!pip3 install torch 
!pip3 install tqdm
!wget -O ./text_files.tar.gz 'https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz' 
!tar -xzf text_files.tar.gz
!pip3 install unidecode

--2018-10-13 21:06:02--  https://piazza.com/redirect/s3?bucket=uploads&prefix=attach%2Fjlifkda6h0x5bk%2Fhzosotq4zil49m%2Fjn13x09arfeb%2Ftext_files.tar.gz
Resolving piazza.com (piazza.com)... 34.200.202.18, 34.237.217.71, 52.20.136.189, ...
Connecting to piazza.com (piazza.com)|34.200.202.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz [following]
--2018-10-13 21:06:02--  https://d1b10bmlvqabco.cloudfront.net/attach/jlifkda6h0x5bk/hzosotq4zil49m/jn13x09arfeb/text_files.tar.gz
Resolving d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)... 13.33.164.102, 13.33.164.12, 13.33.164.9, ...
Connecting to d1b10bmlvqabco.cloudfront.net (d1b10bmlvqabco.cloudfront.net)|13.33.164.102|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1533290 (1.5M) [application/x-gzip]
Saving to: ‘./text_files.tar.gz’

./text_files.tar.gz 100%[

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from tqdm import tqdm

import unidecode
import string
import random
import re
from torch.autograd import Variable
import pdb

## Functions used by the net

In [0]:
# Turn string into list of longs
def char_tensor(string):
  tensor = torch.zeros(len(string)).long()
  for c in range(len(string)):
     tensor[c] = all_characters.index(string[c])
  return Variable(tensor)
  
def random_training_set():    
  chunk = random_chunk()
  inp = char_tensor(chunk[:-1])
  target = char_tensor(chunk[1:])
  return inp, target
 
def random_chunk():
  start_index = random.randint(0, file_len - chunk_len)
  end_index = start_index + chunk_len + 1
  return file[start_index:end_index]

In [0]:
def train(inp, target):
  ## initialize hidden layers, set up gradient and loss 
  # your code here
  hidden = decoder.init_hidden() #initialize hidden
  decoder_optimizer.zero_grad()
  loss = 0
  for c in range(chunk_len):
    output, hidden = decoder(inp[c], hidden) # run the forward pass of your rnn with proper input
    loss += criterion(output, target[c].unsqueeze(0))

  ## calculate backwards loss and step the optimizer (globally)
  # your code here
  ## /
  
  loss.backward()
  decoder_optimizer.step()

  return loss.item() / chunk_len

def evaluate(prime_str='A', predict_len=100, temperature=0.8):
  ## initialize hidden variable, initialize other useful variables 
  hidden = decoder.init_hidden() #initialize hidden
  predicted = prime_str #initialize predicted 
  ## /

  prime_input = char_tensor(prime_str)
  
  # Use priming string to "build up" hidden state
  for p in range(len(prime_str) - 1):
      _, hidden = decoder(prime_input[p], hidden)
  inp = prime_input[-1] #This is an index, not words

  for p in range(predict_len): #Makes the hidden layer for large strings better, then use your input to predict on.
      #Same as taking last letter of prime string if prime str is single
      output, hidden = decoder(inp, hidden) #run your RNN/decoder forward on the input

      # Sample from the network as a multinomial distribution
      output_dist = output.data.view(-1).div(temperature).exp()
      top_i = torch.multinomial(output_dist, 1)[0] #gives you the index of where the most

      ## get character from your list of all characters, add it to your output str sequence, set input
      ## for the next pass through the model
      predicted += all_characters[top_i.item()]  #update predicted
      inp = top_i
      ## /

  return predicted

## The GRU and RNN

In [0]:
class GRU(nn.Module):
  def __init__(self, input_size, hidden_size, n_layers=1 ):
    super(GRU, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.n_layers = n_layers
    
    self.r_t = nn.Sequential(nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias = True), nn.Sigmoid())
    self.z_t = nn.Sequential(nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias = True), nn.Sigmoid())
    self.n_t = nn.Sequential(nn.Linear(self.input_size + self.hidden_size, self.hidden_size, bias = True), nn.Tanh())
    
  def forward(self, input, hidden):
    t = torch.cat((input, hidden), dim = 2) #What is the dimension of W? I.E. what are we concatenating
    r = self.r_t(t)
    z = self.z_t(t)
    
    rh = torch.cat((input, hidden), dim = 2)
    n = self.n_t(rh)
    h = (1-z)*n + z*hidden
    
    return h,h
  

class RNN(nn.Module):
  def __init__(self, input_size, hidden_size, output_size, n_layers=1):
    super(RNN, self).__init__()
    self.input_size = input_size
    self.hidden_size = hidden_size
    self.output_size = output_size
    self.n_layers = n_layers

    # encode using embedding layer
    self.embedding = nn.Embedding(input_size, hidden_size)
    # set up GRU passing in number of layers parameter (nn.GRU)
    self.gru = GRU(input_size, hidden_size)
    # decode output
    self.out = nn.Linear(hidden_size, output_size)
    self.softmax = nn.LogSoftmax(dim=1)

  def forward(self, input_char, hidden):
    # by reviewing the documentation, construct a forward function that properly uses the output
    # of the GRU
    # return output and hidden
    output = self.embedding(input_char).view(1, 1, -1)
    output = F.relu(output)
    output, hidden = self.gru(output, hidden)
    output = self.softmax(self.out(output[0]))
    return output, hidden
      
  def init_hidden(self):
    return Variable(torch.zeros(self.n_layers, 1, self.hidden_size))

# Runs of the Model
 - note that the spec file requests 15 samples per dataset
 - I ran the model on the given LOTR, Alma, and Shakespeare data set, as well as a dataset containing some of Arthur Conan Doyles Sherlock Holmes stories

## Lord of The Rings

In [15]:
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./text_files/lotr.txt').read())
file_len = len(file)
print('file_len =', file_len)

chunk_len = 200
print(random_chunk())

file_len = 2579888
a deep dike with a steep wall on the further 
side. Tom said that it had once been the boundary of a kingdom, but a very 
long lime ago. He seemed to remember something sad about it, and would not 






In [16]:

import time
n_epochs = 2000
print_every = 1000
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
    print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
    all_losses.append(loss_avg / plot_every)
    loss_avg = 0

[175.41901993751526 (1000 50%) 1.9932]
Whinh as wither long gried and mon to me Meres in hirses Rishber thither. Eanter, a foun of a das was  

[351.6507213115692 (2000 100%) 1.6543]
Whiled the grows be me to in the last 
leath. And stiless the sanishave 
fross to to with then 






 



## Alma

In [17]:
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./text_files/alma.txt').read())
file_len = len(file)
print('file_len =', file_len)

chunk_len = 200
print(random_chunk())

file_len = 466656
to him and his brethren.

 And now, these are the words of Ammon to his brethren, which say thus: My brothers and my brethren, behold I say unto you, how great reason have we to rejoice; for could we h


In [18]:
import time
n_epochs = 2000
print_every = 130
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
    print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
    all_losses.append(loss_avg / plot_every)
    loss_avg = 0

[22.865074396133423 (130 6%) 2.2873]
Wh wmaga, fot one chith be pen an ith cow uthirte and ther ake houl of wile laor i ind cond; wo matave 

[45.68302130699158 (260 13%) 2.0276]
Wh.

 whwren in thowl veime helould to gith and in nou and uter nt has and in singed gironto Gong if t 

[68.48992896080017 (390 19%) 1.4951]
Whare now or wereo igtore all beane greoul of on ave ceire iz behe caperen ood ous be worlsed the cass 

[91.40008878707886 (520 26%) 1.6707]
Wh and for it sporbe at or allie.

 reptle and the mich intholige the dy mile the a como or the Noured 

[114.28969955444336 (650 32%) 1.4352]
Whing that ye land he dits reminittert prongoran they el ye to that land the man to fore, and the come 

[137.09772658348083 (780 39%) 1.2309]
Whoreven, and be prelen ye came that thee bristenest you, land of strands, and to cossied yout Pore to 

[160.21630382537842 (910 45%) 1.8157]
Wh, in shallen they hass sain ened, them beeo ard stard, that they had prile sur aganing he will with  

[18

## Shakespeare

In [9]:
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./text_files/tiny_shakespeare.txt').read())
file_len = len(file)
print('file_len =', file_len)

chunk_len = 500
print(random_chunk())

file_len = 1115394
 is dead.
The king my uncle is to blame for this:
God will revenge it; whom I will importune
With daily prayers all to that effect.

Girl:
And so will I.

DUCHESS OF YORK:
Peace, children, peace! the k


In [14]:
import time
n_epochs = 2000
print_every = 130
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
    print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
    all_losses.append(loss_avg / plot_every)
    loss_avg = 0

[90.71150422096252 (500 25%) 2.0692]
Wher to the in fith blour migher,
Af that it lou of wer eace cold shiclon,
As you mo,
An's arcate my e 

[181.56948494911194 (1000 50%) 1.8662]
Whart!
Of plander prold, semay, be one frar,
We a dower lit:
And wast sountion the sunter the deare id 

[272.5564832687378 (1500 75%) 1.7651]
Wher; genent loods
Whought the doth shich, and many hen the plaith
O, the hat, grablers the plight pes 

[363.2676250934601 (2000 100%) 1.8596]
What Luave san, well whuble and at
sispit and it him, his not stood castrent his hither.
Loven of not  



## Sherlock Holmes

In [83]:
all_characters = string.printable
n_characters = len(all_characters)
 
file = unidecode.unidecode(open('./sherlock_holmes.txt').read())
file_len = len(file)
print('file_len =', file_len)

chunk_len = 500
print(random_chunk())

file_len = 143124
ith his knife, and I had to grasp him twice, and got a cut over the knuckles, before I had the upper hand of him. He looked murder out of the only eye he could see with when we had finished, but he listened to reason and gave up the papers. Having got them I let my man go, but I wired full particulars to Forbes this morning. If he is quick enough to catch his bird, well and good. But if, as I shrewdly suspect, he finds the nest empty before he gets there, why, all the better for the government. I


In [85]:
import time
n_epochs = 2000
print_every = 130
plot_every = 10
hidden_size = 100
n_layers = 1
lr = 0.005
 
decoder = RNN(n_characters, hidden_size, n_characters, n_layers)
decoder_optimizer = torch.optim.Adam(decoder.parameters(), lr=lr)
criterion = nn.CrossEntropyLoss()
 
start = time.time()
all_losses = []
loss_avg = 0
 
for epoch in range(1, n_epochs + 1):
  loss_ = train(*random_training_set())       
  loss_avg += loss_

  if epoch % print_every == 0:
    print('[%s (%d %d%%) %.4f]' % (time.time() - start, epoch, epoch / n_epochs * 100, loss_))
    print(evaluate('Wh', 100), '\n')

  if epoch % plot_every == 0:
    all_losses.append(loss_avg / plot_every)
    loss_avg = 0

[61.55309176445007 (130 6%) 2.2109]
Whe isllfonded I and the tolkat at behe wimt heon I andang sothe werctouc a he geasgat whersen I youve 

[123.96553087234497 (260 13%) 2.0102]
Whering and the beas ther the Jatticticher sode you save sine, I the reer ary ailr mured bave yoq to m 

[185.7848024368286 (390 19%) 1.9695]
Whon Ally. A hadiccel and bour heaking a purted the ordenglathed beaded and and the and whof one and w 

[247.6882860660553 (520 26%) 1.9110]
Wh was cank, sept. Hoom, anf am is a proffling of his my a star, and his iver expon my smerpion my sav 

[309.5395288467407 (650 32%) 1.8600]
Why the done nos wabed whow clintied of the wait that. Hor he lun with we the co moveay this the net h 

[371.4916527271271 (780 39%) 1.7035]
Wher, I where beep his cluble me back and Myserply the ofpaned My me thich had everpensice had I help  

[433.74994349479675 (910 45%) 1.7528]
Whare I his over you sepped which shost in to lidly of beer he had storn of a raid. I worched that in  

[495.